Author - [Rohit Shah](www.linkedin.com/in/rohit-shah-00579175)

# Install TFX

- TFX only support the TensorFlow 2 version of Keras.
- restart notebook after this Installation ((Runtime > Restart runtime)

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

!pip install -q -U --use-deprecated=legacy-resolver tfx

# Import

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import pandas as pd
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input


%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [ ]:
# check version 
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.4.1
TFX version: 0.28.0


In [ ]:
%%bash
pwd
mkdir /content/tfx/
mkdir /content/tfx/pipelines
mkdir /content/tfx/metadata
mkdir /content/tfx/logs
mkdir /content/tfx/data
mkdir /content/tfx/serving_model

/content


# Download Data 

we would use famous titanic dataset

In [ ]:
%%bash
wget https://raw.githubusercontent.com/geodra/Titanic-Dataset/master/data/train.csv
wget https://raw.githubusercontent.com/geodra/Titanic-Dataset/master/data/test.csv

--2021-03-10 19:00:50--  https://raw.githubusercontent.com/geodra/Titanic-Dataset/master/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61194 (60K) [text/plain]
Saving to: ‘train.csv’

     0K .......... .......... .......... .......... .......... 83% 3.94M 0s
    50K .........                                             100% 35.7M=0.01s

2021-03-10 19:00:50 (4.61 MB/s) - ‘train.csv’ saved [61194/61194]

--2021-03-10 19:00:50--  https://raw.githubusercontent.com/geodra/Titanic-Dataset/master/data/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sen

In [ ]:
drop_cols = ['PassengerId','Name','Ticket']
df = pd.read_csv("/content/train.csv")
df.drop(drop_cols,axis=1,inplace=True)
df.to_csv("tfx/data/train_new.csv",index=False)

# # test set 
# df = pd.read_csv("/content/test.csv")
# df.drop(drop_cols,axis=1,inplace=True)
# df.to_csv("tfx/data/test_new.csv",index=False)

# Setup Pipeline Path

In [ ]:
##Define all constant
_tfx_root = os.path.join(os.getcwd(), 'tfx');        # Create location ~/tfx
_pipeline_root = os.path.join(_tfx_root, 'pipelines');      # Join ~/tfx/pipelines/
_metadata_db_root = os.path.join(_tfx_root, 'metadata.db');    # Join ~/tfx/metadata.db
_log_root = os.path.join(_tfx_root, 'logs');
_model_root = os.path.join(_tfx_root, 'model');
_data_root = os.path.join(_tfx_root, 'data');
_serving_model_dir = os.path.join(_tfx_root, 'serving_model')
_data_filepath = os.path.join(_data_root, "train_new.csv")

_input_fn_module_file = 'inputfn_trainer.py'
_model_trainer_module_file = 'model_trainer.py'

- if we use default paramete in InteractiveContext than it will use  temporary directory with an ephemeral ML Metadata database instance
- To use your own pipeline root or database, the optional properties `pipeline_root` and `metadata_connection_config` may be passed to
InteractiveContext
- In cloud , this could be path to GCS to store data permanently 

In [ ]:
context = InteractiveContext(pipeline_root=_tfx_root)

# Run indivitual component interactivly 

In [ ]:
_data_root

'/content/tfx/data'

In [ ]:
# ExampleGen
# Input : csv file placed in root folder
# steps Inside:
# 1) split data in train (2/3) & test (1/3)
# 2) convert into TF Record 
# 3) dump data to TFX root for other component to access 
# output : Train & Eval data 

# For us data is already been splitted into train & test so lets define Input config 
# if we want to change the split we had to define output config 
# https://www.tensorflow.org/tfx/guide/examplegen
# from  tfx.proto import example_gen_pb2
# input  = example_gen_pb2.Input(splits=
#                       [
#                        example_gen_pb2.Input.Split(name='train', pattern='train*'), # its  like glob function
#                        example_gen_pb2.Input.Split(name='test', pattern='test*'),
#                       ])

# example_gen = CsvExampleGen(input=external_input(_data_root),input_config=input)
example_gen = CsvExampleGen(input=external_input("/content/tfx/data/"))
context.run(example_gen)

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/content/tfx/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:24811,xor_checksum:1615402850,sum_checksum:1615402850"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Age"
    value {
      float_list {
        value: 22.0
      }
    }
  }
  feature {
    key: "Cabin"
    value {
      bytes_list {
      }
    }
  }
  feature {
    key: "Embarked"
    value {
      bytes_list {
        value: "S"
      }
    }
  }
  feature {
    key: "Fare"
    value {
      float_list {
        value: 7.25
      }
    }
  }
  feature {
    key: "Parch"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Pclass"
    value {
      int64_list {
        value: 3
      }
    }
  }
  feature {
    key: "Sex"
    value {
      bytes_list {
        value: "male"
      }
    }
  }
  feature {
    key: "SibSp"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "Survived"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "Age"
    value {
      float_list {
        value: 38.0
      }
    }
  }
  feature {
    key: "Cabi

In [ ]:
# StaticGen

# Input : example_gen output
# steps Inside:
# 1) compute statistics over dataset using Tensorflow Data validation library
# output :/content/tfx/StatisticsGen/statistics

statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/content/tfx/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# visalize the output of statistics gen 
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [ ]:
# schema gen 
# Input : statistics_gen output
# steps Inside:
# 1) generates a schema based on your data statistics using Tensorflow Data validation library
# 2) The generated schema is best-effort and only tries to infer basic properties of the data.
#    It is expected that you review and modify it as needed.
# 3) Each feature in your dataset shows up as a row in the schema table, alongside its properties. Domain is the set of categorical values
# output :/content/tfx/SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/content/tfx/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
# visalize 
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',FLOAT,required,,-
'Cabin',BYTES,required,,-
'Embarked',STRING,required,,'Embarked'
'Fare',FLOAT,required,single,-
'Parch',INT,required,single,-
'Pclass',INT,required,single,-
'Sex',STRING,required,single,'Sex'
'SibSp',INT,required,single,-
'Survived',INT,required,single,-


In [ ]:
# ExampleValidator
# Input : statistics_gen & schema_gen
# steps Inside:
# 1) The `ExampleValidator` component detects anomalies in your data, based on the expectations defined by the
# schema using Tensorflow Data validation library
# Output: anomalies

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/content/tfx/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [ ]:
context.show(example_validator.outputs['anomalies'])

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [ ]:
# Trainer component
# Input : schema_gen, transformed data & graph, training parameter, tensorflow code
# steps
# 1) train model supports estimator API & Keras API

In [ ]:
%%writefile {_input_fn_module_file}
# Input data to model 

import os
import tensorflow as tf

###############################
##Type casting & add or remove new features
def feature_engg_features(features):
  #Add new features
  features['Pclass'] = tf.strings.as_string(features['Pclass'])
  features['SibSp'] = tf.strings.as_string(features['SibSp'])
  features['Parch'] = tf.strings.as_string(features['Parch'])

  return(features)

#To be called from TF
def feature_engg(features, label):
  #Add new features
  features = feature_engg_features(features)
  
  return(features, label)

def make_input_fn(dir_uri, mode, vnum_epochs = None, batch_size = 512):
    def decode_tfr(serialized_example):
      # 1. define a parser
      features = tf.io.parse_example(
        serialized_example,
        # Defaults are not specified since both keys are required.
        features={
            'Age': tf.io.FixedLenSequenceFeature([], tf.float32, default_value = 0.0,allow_missing=True),
            'Cabin': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
            'Embarked': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True),
            'Fare': tf.io.FixedLenFeature([], tf.float32),
            'Parch': tf.io.FixedLenFeature([], tf.int64),
            'Pclass': tf.io.FixedLenFeature([], tf.int64),
            'Survived': tf.io.FixedLenFeature([], tf.int64),
            'Sex': tf.io.FixedLenFeature([], tf.string),
            'SibSp': tf.io.FixedLenFeature([], tf.int64),
         })

      return features, features['Survived']

    def _input_fn(v_test=False):
      # Get the list of files in this directory (all compressed TFRecord files)
      tfrecord_filenames = tf.io.gfile.glob(dir_uri)

      # Create a `TFRecordDataset` to read these files
      dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

      if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = vnum_epochs # indefinitely
      else:
        num_epochs = 1 # end-of-input after this

      dataset = dataset.batch(batch_size)
      dataset = dataset.prefetch(buffer_size = batch_size)
      
      #Convert TFRecord data to dict
      dataset = dataset.map(decode_tfr)
      #Feature engineering
      dataset = dataset.map(feature_engg)

      if mode == tf.estimator.ModeKeys.TRAIN:
          num_epochs = vnum_epochs # indefinitely
          dataset = dataset.shuffle(buffer_size = batch_size)
      else:
          num_epochs = 1 # end-of-input after this

      dataset = dataset.repeat(num_epochs)       
      
      #Begins - Uncomment for testing only -----------------------------------------------------<
      if v_test == True:
        print(next(dataset.__iter__()))
        
      #End - Uncomment for testing only -----------------------------------------------------<
      return dataset
    return _input_fn

Writing inputfn_trainer.py


In [ ]:
import inputfn_trainer
import importlib
importlib.reload(inputfn_trainer)

import inputfn_trainer as ift
#Test dataset read + Feat Engg function's - output's CSV + Feature engg columns
train_file = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train/*')
fn_d = ift.make_input_fn(dir_uri = train_file,
                    mode = tf.estimator.ModeKeys.EVAL,
                    # vnum_epochs = 1,
                    batch_size = 10)

fn_d(v_test=True)

({'Age': <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[22.],
       [38.],
       [35.],
       [ 0.],
       [27.],
       [58.],
       [20.],
       [39.],
       [14.],
       [55.]], dtype=float32)>, 'Cabin': <tf.Tensor: shape=(10, 1), dtype=string, numpy=
array([[b''],
       [b'C85'],
       [b'C123'],
       [b''],
       [b''],
       [b'C103'],
       [b''],
       [b''],
       [b''],
       [b'']], dtype=object)>, 'Embarked': <tf.Tensor: shape=(10, 1), dtype=string, numpy=
array([[b'S'],
       [b'C'],
       [b'S'],
       [b'Q'],
       [b'S'],
       [b'S'],
       [b'S'],
       [b'S'],
       [b'S'],
       [b'S']], dtype=object)>, 'Fare': <tf.Tensor: shape=(10,), dtype=float32, numpy=
array([ 7.25  , 71.2833, 53.1   ,  8.4583, 11.1333, 26.55  ,  8.05  ,
       31.275 ,  7.8542, 16.    ], dtype=float32)>, 'Parch': <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'0', b'0', b'0', b'0', b'2', b'0', b'0', b'5', b'0', b'0'],
      dtype=object)>, 'Pclass': 

<RepeatDataset shapes: ({Age: (None, None), Cabin: (None, None), Embarked: (None, None), Fare: (None,), Parch: (None,), Pclass: (None,), Sex: (None,), SibSp: (None,), Survived: (None,)}, (None,)), types: ({Age: tf.float32, Cabin: tf.string, Embarked: tf.string, Fare: tf.float32, Parch: tf.string, Pclass: tf.string, Sex: tf.string, SibSp: tf.string, Survived: tf.int64}, tf.int64)>

In [ ]:
%%writefile {_model_trainer_module_file}

import tensorflow as tf
import tensorflow.keras as keras
import inputfn_trainer as ift

from tfx.components.trainer.fn_args_utils import FnArgs
print(tf.__version__)

device = "gpu"

if device == "tpu":
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  # This is the TPU initialization code that has to be at the beginning.
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
else:
  strategy = tf.distribute.MultiWorkerMirroredStrategy()

#Create model
params_default = {
    'lr' : 0.001,
    'beta_1' : 0.99,
    'beta_2' : 0.999,
    'epsilon' : 1e-08,
    'decay' : 0.01,
    'hidden_layers' : 1
}

def create_feature_cols():
    #Keras format features
    k_pclass = tf.keras.Input(name='Pclass', shape=(1,), dtype=tf.string)
    k_sex = tf.keras.Input(name='Sex', shape=(1,), dtype=tf.string)
    k_age = tf.keras.Input(name='Age', shape=(1,), dtype=tf.float32)
    k_sibsp = tf.keras.Input(name='SibSp', shape=(1,), dtype=tf.string)
    k_parch = tf.keras.Input(name='Parch', shape=(1,), dtype=tf.string)
    k_fare= tf.keras.Input(name='Fare', shape=(1,), dtype=tf.float32)
    k_cabin= tf.keras.Input(name='Cabin', shape=(1,), dtype=tf.string)
    k_embarked= tf.keras.Input(name='Embarked', shape=(1,), dtype=tf.string)

    keras_dict_input = {'Pclass': k_pclass, 'Sex': k_sex, 'Age' : k_age,
                        'SibSp': k_sibsp, 'Parch': k_parch,
                        'Fare': k_fare, 'Cabin': k_cabin, 'Embarked' : k_embarked
                        }

    return({'K' : keras_dict_input})

def create_keras_model(feature_cols, params = params_default):
    METRICS = [
            keras.metrics.BinaryCrossentropy(name="binary_crossentropy")
    ]

    # define input layer to model 
    input_feats = []
    for inp in feature_cols['K'].keys():
      input_feats.append(feature_cols['K'][inp])

    cat_pclass = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['1','2','3'], mask_token=None)(feature_cols['K']['Pclass'])
    cat_pclass = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=3)(cat_pclass)

    cat_psex = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['male','female'], mask_token=None)(feature_cols['K']['Sex'])
    cat_psex = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=2)(cat_psex)

    cat_sibSp= tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['0','1','2','3','4','5','8'], mask_token=None)(feature_cols['K']['SibSp'])
    cat_sibSp = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=7)(cat_sibSp)

    cat_parch= tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['0','1','2','3','4','5','6'], mask_token=None)(feature_cols['K']['Parch'])
    cat_parch = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=7)(cat_parch)

    cat_embarked= tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['S','C','Q'], mask_token=None)(feature_cols['K']['Embarked'])
    cat_embarked = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=3)(cat_embarked)

    ###Create MODEL
    ####Concatenate all features( Numerical input )
    x_input_numeric = tf.keras.layers.concatenate([
                    feature_cols['K']['Age'], feature_cols['K']['Fare']
                    ])
    #DEEP - This Dense layer connects to input layer - Numeric Data
    x_numeric = tf.keras.layers.Dense(32, activation='relu', kernel_initializer="he_uniform")(x_input_numeric)
    x_numeric = tf.keras.layers.BatchNormalization()(x_numeric)

    ####Concatenate all Categorical features( Categorical converted )
    x_input_categ = tf.keras.layers.concatenate([
                    cat_pclass, cat_psex,cat_sibSp,cat_parch,cat_embarked
                    ])
    ####Concatenate both Wide and Deep layers
    x = tf.keras.layers.concatenate([x_input_categ, x_numeric])

    for l_ in range(params['hidden_layers']):
        x = tf.keras.layers.Dense(32, activation='relu', kernel_initializer="he_uniform",
                                  activity_regularizer=tf.keras.regularizers.l2(0.00001))(x)
        x = tf.keras.layers.BatchNormalization()(x)

    #Final Layer
    out = tf.keras.layers.Dense(1, activation='relu')(x)
    model = tf.keras.Model(input_feats, out)

    #Set optimizer
    opt = tf.keras.optimizers.Adam(lr= params['lr'], beta_1=params['beta_1'], 
                                        beta_2=params['beta_2'], epsilon=params['epsilon'])

    #Compile model
    model.compile(loss='binary_crossentropy',  optimizer=opt, metrics = METRICS)

    #Print Summary
    print(model.summary())
    return model

def keras_train_and_evaluate(model, train_dataset, validation_dataset, epochs=100):
  #Add callbacks
  reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=5, min_lr=0.00001, verbose = 1)
  
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

  print("Training model")
  #Train and Evaluate
  out = model.fit(train_dataset, 
                  validation_data = validation_dataset,
                  epochs=epochs,
                  # validation_steps = 3,   ###Keep this none for running evaluation on full EVAL data every epoch
                  steps_per_epoch = 100,   ###Has to be passed - Cant help it :) [ Number of batches per epoch ]
                  callbacks=[reduce_lr, #modelsave_callback, #tensorboard_callback, 
                             keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True, verbose=True)]
                  )

  return model

##Main function called by TFX
def run_fn(fn_args: FnArgs):
  #Create dataset input functions
  train_dataset = ift.make_input_fn(dir_uri = fn_args.train_files,
                      mode = tf.estimator.ModeKeys.TRAIN,
                      batch_size = 128)()

  validation_dataset = ift.make_input_fn(dir_uri = fn_args.eval_files,
                      mode = tf.estimator.ModeKeys.EVAL,
                      batch_size = 512)()

  print(params_default)
  m_ = create_keras_model(params = params_default, feature_cols = create_feature_cols())
  tf.keras.utils.plot_model(m_, show_shapes=True, rankdir="LR")

  m_ = keras_train_and_evaluate(m_, train_dataset, validation_dataset, fn_args.custom_config['epochs'])
  # save_model(m_, fn_args.serving_model_dir)

Writing model_trainer.py


In [ ]:
trainer = Trainer(
    module_file=os.path.abspath(_model_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=example_gen.outputs['examples'],
    train_args=trainer_pb2.TrainArgs(),
    eval_args=trainer_pb2.EvalArgs(),
    custom_config=({"epochs": 1})
    )

context.run(trainer)

2.4.1
INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
{'lr': 0.001, 'beta_1': 0.99, 'beta_2': 0.999, 'epsilon': 1e-08, 'decay': 0.01, 'hidden_layers': 1}
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Pclass (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
Sex (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
SibSp (InputLayer)              [(None, 1)]          0            

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['Survived'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


100/100 [==============================] - 3s 13ms/step - loss: 3.4515 - binary_crossentropy: 3.4513 - val_loss: 2.7998 - val_binary_crossentropy: 2.7994


RuntimeError: ignored

In [ ]:
# cat_day_p = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['1','2','3'], mask_token=None)
# cat_day = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=3)
# cat_day(cat_day_p(list(data_.Pclass.astype('string').values)))

In [ ]:
data_.columns

In [ ]:
data_.info()

In [ ]:
import pandas as pd
data_ = pd.read_csv("/content/tfx/data/train_new.csv")

In [ ]:
data_.columns

In [ ]:
data_.Embarked.isnull().sum()

In [ ]:
data_.columns